In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Real Personal Consumption Expenditures (PCS)


In [21]:
pce = pd.read_csv("../data/PERSONAL_EXPENDITURE.csv")
save = pd.read_csv("../data/SAVING_RATE.csv")
credit = pd.read_csv("../data/CREDIT_OWNED.csv")

print(pce.head())
print(save.head())
print(credit.head())
print(pce.columns, save.columns, credit.columns)


  observation_date   PCEC96
0       2007-01-01  11181.0
1       2007-02-01  11178.2
2       2007-03-01  11190.7
3       2007-04-01  11201.5
4       2007-05-01  11218.0
  observation_date  PSAVERT
0       1959-01-01     11.3
1       1959-02-01     10.6
2       1959-03-01     10.3
3       1959-04-01     11.2
4       1959-05-01     10.6
  observation_date  TOTALSL
0       1943-01-01  6577.83
1       1943-02-01  6463.04
2       1943-03-01  6234.21
3       1943-04-01  6125.75
4       1943-05-01  5936.26
Index(['observation_date', 'PCEC96'], dtype='object') Index(['observation_date', 'PSAVERT'], dtype='object') Index(['observation_date', 'TOTALSL'], dtype='object')


In [23]:
#Standardize column names

# --- PCE ---
pce.columns = [c.strip().upper() for c in pce.columns]
pce = pce.rename(columns={pce.columns[0]: "DATE", pce.columns[1]: "PCE_REAL"})
pce["DATE"] = pd.to_datetime(pce["DATE"])
pce = pce.sort_values("DATE")

# --- SAVING RATE ---
save.columns = [c.strip().upper() for c in save.columns]
save = save.rename(columns={save.columns[0]: "DATE", save.columns[1]: "SAVING_RATE"})
save["DATE"] = pd.to_datetime(save["DATE"])
save = save.sort_values("DATE")

# --- CREDIT ---
credit.columns = [c.strip().upper() for c in credit.columns]
credit = credit.rename(columns={credit.columns[0]: "DATE", credit.columns[1]: "CONSUMER_CREDIT"})
credit["DATE"] = pd.to_datetime(credit["DATE"])
credit = credit.sort_values("DATE")

In [24]:
print(pce.head())
print(save.head())
print(credit.head())

        DATE  PCE_REAL
0 2007-01-01   11181.0
1 2007-02-01   11178.2
2 2007-03-01   11190.7
3 2007-04-01   11201.5
4 2007-05-01   11218.0
        DATE  SAVING_RATE
0 1959-01-01         11.3
1 1959-02-01         10.6
2 1959-03-01         10.3
3 1959-04-01         11.2
4 1959-05-01         10.6
        DATE  CONSUMER_CREDIT
0 1943-01-01          6577.83
1 1943-02-01          6463.04
2 1943-03-01          6234.21
3 1943-04-01          6125.75
4 1943-05-01          5936.26


In [26]:
# Pulling Extra indicators from FRED
cpi_url = "https://fred.stlouisfed.org/graph/fredgraph.csv?id=CPIAUCSL"
ffr_url = "https://fred.stlouisfed.org/graph/fredgraph.csv?id=FEDFUNDS"
dpi_url = "https://fred.stlouisfed.org/graph/fredgraph.csv?id=DSPIC96"

cpi = pd.read_csv(cpi_url)
ffr = pd.read_csv(ffr_url)
dpi = pd.read_csv(dpi_url)

cpi = cpi.rename(columns={"observation_date": "DATE", "CPIAUCSL": "CPI"})
ffr = ffr.rename(columns={"observation_date": "DATE", "FEDFUNDS": "FEDFUNDS"})
dpi = dpi.rename(columns={"observation_date": "DATE", "DSPIC96": "REAL_DPI"})

cpi["DATE"] = pd.to_datetime(cpi["DATE"])
ffr["DATE"] = pd.to_datetime(ffr["DATE"])
dpi["DATE"] = pd.to_datetime(dpi["DATE"])

In [27]:
# Merge everything into one master dataset

df = pce.merge(save, on="DATE", how="outer")
df = df.merge(credit, on="DATE", how="outer")
df = df.merge(cpi, on="DATE", how="outer")
df = df.merge(ffr, on="DATE", how="outer")
df = df.merge(dpi, on="DATE", how="outer")

df = df.sort_values("DATE").reset_index(drop=True)

print(df.shape)
df.head()

(997, 7)


,DATE,PCE_REAL,SAVING_RATE,CONSUMER_CREDIT,CPI,FEDFUNDS,REAL_DPI
0,1943-01-01,NaN,NaN,6577.83,NaN,NaN,NaN
1,1943-02-01,NaN,NaN,6463.04,NaN,NaN,NaN
2,1943-03-01,NaN,NaN,6234.21,NaN,NaN,NaN
3,1943-04-01,NaN,NaN,6125.75,NaN,NaN,NaN
4,1943-05-01,NaN,NaN,5936.26,NaN,NaN,NaN


In [28]:
# Cleaning Missing Values

for col in df.columns:
    if col != "DATE":
        df[col] = pd.to_numeric(df[col], errors="coerce")

print(df.isna().sum())

DATE                 0
PCE_REAL           770
SAVING_RATE        194
CONSUMER_CREDIT      2
CPI                 49
FEDFUNDS           138
REAL_DPI           193
dtype: int64
